In [15]:
# import Sent2textDataset
import torch
from sklearn.metrics import precision_score
from torch.utils.data import DataLoader

from clip.evaluate.utils import (
    load_weights_only,
)

In [8]:
def train_model(model, train_loader, val_loader, loss, optimizer, num_epochs):
    loss_history = []
    train_history = []
    val_history = []
    val_loss_hist = []
    metric_y_val = metric_p_val = None
    
#     scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,
#                                                   T_0=40, 
#                                                   T_mult=2,
#                                                   eta_min=1e-9)
    for epoch in range(num_epochs):
        
        print(epoch)
        model.train()
        
        correct_samples = 0
        total_samples = 0
        loss_accum = 0
        for i_step, (imgs, text_vect, att),label ,_  in enumerate(train_loader):
                
                imgs_gpu = imgs.to(device=device)
                text_vect_gpu = text_vect.to(device=device)
                att_gpu = att.to(device=device)
                
                y_gpu = label.to(device=device)
                
                _, prediction = model(x_gpu) # logits_per_text
                loss_value = loss(prediction, y_gpu.reshape(-1))
                _, preds = torch.max(prediction, 1)
                

                optimizer.zero_grad()
                loss_value.backward()
                optimizer.step()
                
                
                if i_step == 0 and epoch == 0:
                    metric_y = y_gpu.reshape((-1,)).cpu().numpy()
                    metric_p = preds.cpu().numpy()
                else:
                    metric_y = np.concatenate((metric_y, y_gpu.reshape((-1,)).cpu().numpy()))
                    metric_p = np.concatenate((metric_p, preds.cpu().numpy())) 
                    
                correct_samples += torch.sum(preds == y_gpu.reshape((-1,)))
                loss_accum += loss_value
                
                total_samples += y_gpu.shape[0]
                del x_gpu
                del y_gpu
                #gc.collect()
        
        
        ave_loss = loss_accum / (i_step + 1)
        train_accuracy = correct_samples / total_samples
#         writer.add_scalar("Loss/train", ave_loss, epoch)
#         writer.add_scalar("Acc/train", train_accuracy, epoch)
#         writer.add_scalar("F1/train", f1_score(metric_y,metric_p), epoch)

        val_accuracy, loss_val,metric_y_val, metric_p_val = compute_valid(model, val_loader, loss, epoch,metric_y_val, metric_p_val)
#         writer.add_scalar("Loss/valid", loss_val, epoch)
#         writer.add_scalar("Acc/valid", val_accuracy, epoch)
#         writer.add_scalar("F1/valid", f1_score(metric_y_val,metric_p_val), epoch)
        
#         writer.add_scalar("Lr/epoch", scheduler.get_last_lr()[-1], epoch)
        scheduler.step(epoch)
        
        loss_history.append(float(ave_loss))
        train_history.append(train_accuracy)
        val_history.append(val_accuracy)
        val_loss_hist.append(loss_val)

        print("Average loss: %f, Val loss: %f, Train accuracy: %f, Val accuracy: %f, Train F1: %f,Val F1: %f" % (ave_loss,loss_val, train_accuracy, val_accuracy,f1_score(metric_y,metric_p), f1_score(metric_y_val,metric_p_val)))
        
#         print('Epoch:', epoch, 'LR:', scheduler.get_last_lr())

def compute_valid(model, loader, loss,epoch, metric_y= None, metric_p=None):
    model.eval()
    with torch.no_grad():
        correct_samples = 0
        total_samples = 0
        loss_accum = 0
        
        for i_step, (x, y) in enumerate(loader):
            x_gpu = x.to(device=device, dtype=torch.float)
            y_gpu = y.to(device=device,)

            _, prediction = model(x_gpu)
            loss_value = loss(prediction, y_gpu.reshape((-1,)))
            _, preds = torch.max(prediction, 1)
            
            
            if i_step == 0 and epoch == 0:
                metric_y = y_gpu.reshape((-1,)).cpu().numpy()
                metric_p = preds.cpu().numpy()
            else:
                metric_y = np.concatenate((metric_y, y_gpu.reshape((-1,)).cpu().numpy()))
                metric_p = np.concatenate((metric_p, preds.cpu().numpy())) 
            
            
            correct_samples += torch.sum(preds == y_gpu.reshape((-1,)))
            total_samples += y_gpu.shape[0]
            loss_accum += loss_value

            del x_gpu
            del y_gpu
                
        loss_val = loss_accum / (i_step + 1)
        val_accuracy = correct_samples / total_samples
        return val_accuracy, loss_val, metric_y, metric_p

In [9]:
from CustomDataset import Sent2textDataset


In [13]:
path_t_csv = "data/60k_data_preproc.csv" #pd.read_csv("data/60k_data_preproc.csv")[:1000]
path_i_json = "data/images.json"
path_i_folder = "data/images"


# в csv file для каждого касса должен быть массив
model, args = load_weights_only("ViT-B/32-small",seq_length = 15)

ds = Sent2textDataset(path_t_csv,
                      path_i_json,path_i_folder,
                      down_data = False,
                      n_classes = 5,args = args)

/home/viktor.kumpan/.miniconda3/lib/python3.8/site-packages/torchvision/transforms/transforms.py:257: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
Some weights of the model checkpoint at sberbank-ai/rugpt3small_based_on_gpt2 were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
5462418it [00:11, 456012.02it/s]


data/images
From 50971 sample folder hasn't 50038


In [40]:
model = model.cuda().float().eval()

In [18]:
dl = DataLoader(ds, batch_size = 4)

In [41]:
for (imgs, text_vect, att),label,_ in dl:
    pr = model(
        img_input={"x": torch.squeeze(imgs,1)},
        text_input={"x": text_vect, "attention_mask":att})
    break

In [44]:
pr[1].shape

torch.Size([20, 4])